In [60]:
import os
import re
import pickle
from typing import List, Dict, Any, Tuple, Optional
from pathlib import Path
from dataclasses import dataclass

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [61]:
@dataclass
class SearchResult:
    """검색 결과 분석을 위한 데이터 클래스"""
    strategy_name: str
    query: str
    documents: List[Document]
    keyword_scores: List[float]
    total_docs: int
    relevant_docs: int
    top_3_relevance: float


class DocumentSplittingStrategy:
    """문서 분할 전략의 기본 클래스"""
    
    def __init__(self, document_path: str):
        self.document_path = document_path
        self.raw_text = self._load_document()
        
    def _load_document(self) -> str:
        """원본 문서 로드"""
        with open(self.document_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def split_documents(self) -> List[Document]:
        """문서 분할 (각 전략에서 구현)"""
        raise NotImplementedError
    
    def get_strategy_name(self) -> str:
        """전략 이름 반환"""
        raise NotImplementedError

In [63]:
class SemanticBasedSplitter(DocumentSplittingStrategy):
    """의미 기반 분할 전략"""
    
    def get_strategy_name(self) -> str:
        return "semantic_based"
    
    def split_documents(self) -> List[Document]:
        """의미 단위로 문서 분할"""
        documents = []
        
        # 헤더 기반 분할
        header_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=[
                ("#", "Header 1"),
                ("##", "Header 2"),
                ("###", "Header 3"),
            ]
        )
        header_docs = header_splitter.split_text(self.raw_text)
        
        # 각 섹션을 의미 단위로 분할
        for doc in header_docs:
            semantic_chunks = self._split_by_semantic_units(doc.page_content)
            
            for i, chunk in enumerate(semantic_chunks):
                metadata = doc.metadata.copy()
                metadata.update({
                    "chunk_index": i,
                    "source_strategy": "semantic_based",
                    "semantic_score": self._calculate_semantic_score(chunk)
                })
                documents.append(Document(page_content=chunk, metadata=metadata))
        
        return documents
    
    def _split_by_semantic_units(self, content: str) -> List[str]:
        """의미 단위로 분할"""
        # 단락, 문장, 의미 구조를 고려한 분할
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1200,  # 더 큰 청크로 문맥 보존
            chunk_overlap=200,
            separators=["\n\n\n", "\n\n", "\n", ". ", "? ", "! ", ", "]
        )
        return text_splitter.split_text(content)
    
    def _calculate_semantic_score(self, content: str) -> float:
        """의미적 완성도 점수 계산"""
        # 간단한 휴리스틱으로 의미적 완성도 측정
        sentences = re.split(r'[.!?]', content)
        complete_sentences = [s for s in sentences if len(s.strip()) > 10]
        
        # 완성된 문장 비율, 길이 적정성 등을 고려
        completeness = len(complete_sentences) / len(sentences) if sentences else 0
        length_score = min(1.0, len(content) / 500)  # 500자 기준 정규화
        
        return (completeness + length_score) / 2
      

In [82]:
class MultiLevelSplittingStrategy:
    """다중 레벨 분할 전략 - 큰 단위 + 작은 단위 문서 생성"""
    
    def __init__(self, document_path: str):
        self.document_path = document_path
        self.raw_text = self._load_document()
        
    def _load_document(self) -> str:
        with open(self.document_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def split_documents(self) -> List[Document]:
        """다중 레벨로 문서 분할"""
        documents = []
        
        # 헤더 기반 분할
        header_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=[
                ("#", "Header 1"),
                ("##", "Header 2"),
                ("###", "Header 3"),
                ("####", "Header 4")
            ]
        )
        header_docs = header_splitter.split_text(self.raw_text)
        
        # 계층별로 문서 그룹핑
        hierarchy_groups = self._group_by_hierarchy(header_docs)
        
        # 각 레벨별 문서 생성
        for level, group_docs in hierarchy_groups.items():
            documents.extend(self._create_level_documents(group_docs, level))
        
        return documents
    
    def _group_by_hierarchy(self, header_docs: List[Document]) -> Dict[str, List[Document]]:
        """계층별로 문서 그룹핑"""
        groups: Dict[str, List[Document]] = {
            "major": [],     # 대제목 레벨 (H1)
            "medium": [],    # 중제목 레벨 (H2)
            "minor": [],     # 소제목 레벨 (H3, H4)
        }
        
        for doc in header_docs:
            # 가장 높은 헤더 레벨 확인
            if "Header 1" in doc.metadata:
                groups["major"].append(doc)
            elif "Header 2" in doc.metadata:
                groups["medium"].append(doc)
            else:
                groups["minor"].append(doc)
        
        return groups
    
    def _create_level_documents(self, docs: List[Document], level: str) -> List[Document]:
        """레벨별 문서 생성"""
        level_documents = []
        
        # 레벨별 청크 크기 설정
        chunk_sizes = {
            "major": 2000,    # 대제목: 큰 컨텍스트 보존
            "medium": 1200,   # 중제목: 중간 크기
            "minor": 800      # 소제목: 세밀한 분할
        }
        
        chunk_size = chunk_sizes.get(level, 1000)
        
        for doc in docs:
            # 계층적 제목 생성
            title_hierarchy = self._build_title_hierarchy(doc.metadata)
            
            # 컨텍스트 강화
            enhanced_content = self._enhance_with_context(doc.page_content, title_hierarchy, level)
            
            # 청킹
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=200,
                separators=["\n\n", "\n", ". ", "? ", "! ", ", "]
            )
            chunks = text_splitter.split_text(enhanced_content)
            
            for i, chunk in enumerate(chunks):
                metadata = doc.metadata.copy()
                metadata.update({
                    "chunk_index": i,
                    "hierarchy_level": level,
                    "title_hierarchy": title_hierarchy,
                    "source_strategy": f"multi_level_{level}",
                    "chunk_size": len(chunk),
                    "contains_pns": self._check_pns_context(chunk, title_hierarchy)
                })
                level_documents.append(Document(page_content=chunk, metadata=metadata))
        
        return level_documents
    
    def _build_title_hierarchy(self, metadata: Dict) -> str:
        """제목 계층 구조 생성"""
        hierarchy_parts = []
        for level in range(1, 5):
            header_key = f"Header {level}"
            if header_key in metadata and metadata[header_key]:
                hierarchy_parts.append(metadata[header_key].strip())
        return " > ".join(hierarchy_parts) if hierarchy_parts else "Unknown"
    
    def _enhance_with_context(self, content: str, title_hierarchy: str, level: str) -> str:
        """레벨별 컨텍스트 강화"""
        # PNS 관련 섹션인지 확인
        is_pns_section = "PNS" in title_hierarchy.upper() or "PAYMENT NOTIFICATION" in title_hierarchy.upper()
        
        context_info = f"[계층]: {title_hierarchy}\n"
        
        if is_pns_section:
            context_info += "[PNS 관련]: 이 내용은 PNS(Payment Notification Service) 결제알림서비스와 관련됩니다.\n"
        
        context_info += f"[레벨]: {level}\n\n"
        
        return context_info + content
    
    def _check_pns_context(self, content: str, title_hierarchy: str) -> bool:
        """PNS 컨텍스트 여부 확인"""
        content_upper = content.upper()
        hierarchy_upper = title_hierarchy.upper()
        
        return ("PNS" in hierarchy_upper or 
                "PAYMENT NOTIFICATION" in hierarchy_upper or
                "PNS" in content_upper or
                "PAYMENT NOTIFICATION" in content_upper)


In [ ]:
class SmartRetriever:
    """스마트 검색기 - 키워드 우선순위 기반"""
    
    def __init__(self, documents: List[Document], embedding_model_name: str = "bge-m3:latest"):
        self.documents = documents
        self.embedding_model_name = embedding_model_name
        self.vector_store = None
        self.bm25_retriever = None
        self.ensemble_retriever = None
        
    def build_retrievers(self):
        """검색기 구축"""
        print(f"🔧 검색기 구축 중... (문서 수: {len(self.documents)})")
        
        # Vector store 구축
        embeddings = OllamaEmbeddings(model=self.embedding_model_name)
        self.vector_store = FAISS.from_documents(self.documents, embeddings)
        
        # BM25 검색기 구축
        self.bm25_retriever = BM25Retriever.from_documents(
            self.documents,
            bm25_params={"k1": 1.5, "b": 0.75}
        )
        self.bm25_retriever.k = 20
        
        # Vector 검색기
        vector_retriever = self.vector_store.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 20, "fetch_k": 50, "lambda_mult": 0.7}
        )
        
        # 앙상블 검색기
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, vector_retriever],
            weights=[0.6, 0.4]  # BM25에 더 높은 가중치
        )
        
        print("✅ 검색기 구축 완료")
    
    def get_retriever(self):
        return self.ensemble_retriever
    
    def smart_search(self, query: str, max_results: int = 10) -> List[Document]:
        """스마트 검색 - 키워드 우선순위 적용"""
        if not self.ensemble_retriever:
            raise ValueError("검색기가 구축되지 않았습니다. build_retrievers()를 먼저 호출하세요.")
        
        # 1. 앙상블 검색으로 더 많은 후보 검색
        raw_results = self.ensemble_retriever.invoke(query)
        
        # 2. 키워드 기반 필터링 및 점수 계산
        scored_results = self._score_documents(query, raw_results)
        
        # 3. 점수순 정렬
        scored_results.sort(key=lambda x: x[0], reverse=True)
        
        # 4. 상위 결과 반환
        return [doc for score, doc in scored_results[:max_results]]
    
    def _score_documents(self, query: str, documents: List[Document]) -> List[Tuple[float, Document]]:
        """문서 점수 계산"""
        scored_docs = []
        query_keywords = self._extract_query_keywords(query)
        
        for doc in documents:
            score = 0.0
            content_lower = doc.page_content.lower()
            
            # 1. 키워드 매칭 점수 (가장 중요)
            keyword_matches = 0
            for keyword in query_keywords:
                if keyword.lower() in content_lower:
                    keyword_matches += 1
                    # 정확한 매칭에 높은 점수
                    if keyword.lower() == keyword.lower():  # 완전 일치
                        score += 10
                    else:
                        score += 5
            
            # 2. 키워드 밀도 점수
            density = doc.metadata.get('keyword_density', 0)
            score += density * 20
            
            # 3. 전략별 보너스 점수
            strategy = doc.metadata.get('source_strategy', '')
            if 'keyword' in strategy:
                score += 5
            
            # 4. 위치 점수 (문서 앞부분에 키워드가 있으면 가점)
            first_half = content_lower[:len(content_lower)//2]
            if any(kw.lower() in first_half for kw in query_keywords):
                score += 8
            
            # 5. 길이 적정성 점수
            doc_length = len(doc.page_content.split())
            if 50 <= doc_length <= 300:  # 적정 길이
                score += 3
            
            doc.metadata['search_score'] = score
            doc.metadata['keyword_matches'] = keyword_matches
            scored_docs.append((score, doc))
        
        return scored_docs
    
    def _extract_query_keywords(self, query: str) -> List[str]:
        """쿼리에서 중요 키워드 추출"""
        # 기술 용어 패턴
        tech_patterns = [
            r'\b[A-Z]{2,}\b',  # PNS, API 등
            r'\b[a-z]+[A-Z][a-zA-Z]*\b',  # purchaseState 등
        ]
        
        keywords = []
        for pattern in tech_patterns:
            keywords.extend(re.findall(pattern, query))
        
        # 한글 키워드 추가
        korean_keywords = ['메시지', '규격', '값', '구성', '상태', '결제', '서버']
        for keyword in korean_keywords:
            if keyword in query:
                keywords.append(keyword)
        
        return list(set(keywords))

In [85]:
document_path = "data/dev_center_guide_allmd_touched.md"
docsplit = MultiLevelSplittingStrategy(document_path)
docs = docsplit.split_documents()

idx = 0
res = 0
for doc in docs:
  idx += 1
  if 'PNS' in doc.page_content:
    res += 1
    print(doc.metadata)
    print(doc.page_content)
    print("-" * 100)
    
print(f"res: {res} / len(docs): {len(docs)}")

{'Header 1': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'chunk_index': 0, 'hierarchy_level': 'major', 'title_hierarchy': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'source_strategy': 'multi_level_major', 'chunk_size': 1223, 'contains_pns': True}
[계층]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
[레벨]: major

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* 

In [78]:
document_path = "data/dev_center_guide_allmd_touched.md"

semantic_based_splitter = SemanticBasedSplitter(document_path)
docs = semantic_based_splitter.split_documents()

idx = 0
res = 0
for doc in docs:
  idx += 1
  if 'PNS' in doc.page_content:
    res += 1
    print("--> idx: ", idx)
    print(doc.metadata)
    print(doc.page_content)
    print("-" * 100)

print(f"res: {res} / len(docs): {len(docs)}")

--> idx:  2
{'Header 1': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'chunk_index': 0, 'source_strategy': 'semantic_based', 'semantic_score': 0.9230769230769231}
원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App

In [79]:
retriever = SmartRetriever(docs)
retriever.build_retrievers()

res_doc = retriever.smart_search("PNS 메시지 서버 규격의 purchaseState는 어떤 값으로 구성되나요?", max_results=10)  

### print result ###
idx = 0
for doc in res_doc:
  idx += 1
  print(f"--> idx: {idx}")  
  print(doc.page_content)
  print("-" * 100)

print(f"res_doc: {len(res_doc)}")

🔧 검색기 구축 중... (문서 수: 627)
✅ 검색기 구축 완료
--> idx: 1
\  
#### PNS 메시지 규격 변경  <a href="#id-09.-pns" id="id-09.-pns"></a>  
* 원화 외의 통화를 지원하기 위하여 결제금액(price)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제금액의 통화코드(priceCurrencyCode)를 추가하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제수단별금액(amount)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 응답 규격에 purchaseToken, environment, marketCode 필드가 추가되었습니다.&#x20;  
상세한 규격은 PNS 메시지 상세 변경 내역에서 확인 할 수 있습니다. &#x20;
----------------------------------------------------------------------------------------------------
--> idx: 2
| 변경           | 구매취소 내역조회           | /v2/purchase/voided-purchases/{packageName}                                         | /v6/apps/{packageName}/voided-purchases                                                          | <p>API 사용성 향상을 위하여 구매취소내역 조회기준(startTime, endTime)을 구매일시에서 구매취소일시로 변경</p><p>응답규격에 purchaseToken, marketCode 추가</p>        |
| 변경           | 구매상품 소비             | /v2/purchase/consume/{purchaseId}/{packageName}                 

In [80]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# RAG 체인 구성 요소
# embedding_model = OllamaEmbeddings(model=fixed_model_name)
llm = ChatOllama(model="exaone3.5:latest", temperature=0.3)

# 검색기 설정
retriever = retriever.get_retriever()
# docs.as_retriever(
#     search_type="mmr",
#     search_kwargs={"k": 20, "fetch_k": 50, "lambda_mult": 0.7}
# )

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 반드시 knowledge base의 검색 결과를 토대로 답변해 주세요. 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요
6. 원스토어 이외의 정보는 답변하지 마세요.
7. url 등을 노출할때 onestore가 포함되지 않는 경우 노출되지 않아야 합니다.

컨텍스트: {context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [81]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 초기화는 어떻게 하나요?",
    # "정기결제 구현 방법을 알려주세요",
    # "PNS(Payment Notification Service) 설정 방법은?",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    # "원스토어의 PNS란 무엇이고 어떻게 구현하나요? 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요",
    # "PNS(Payment Notification Service)는 무엇이고 메세지 규격은 어떻게 됩니까?.",
    "PNS 메시지 서버 규격의 purchaseState는 어떤 값으로 구성되나요?"
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: PNS 메시지 서버 규격의 purchaseState는 어떤 값으로 구성되나요?
------------------------------------------------------------
PNS 메시지 서버 규격의 `purchaseState`는 다음과 같은 값으로 구성됩니다:

- `PURCHASED`: 구매 완료 상태
- `PENDING`: 구매 대기 상태
- `REFUNDED`: 환불 완료 상태
- `REFUNDING`: 환불 중 상태
- `SUBSCRIPTION_CANCELLED`: 구독 해지 상태
- `SUBSCRIPTION_PENDING`: 구독 대기 상태
- `SUBSCRIPTION_ACTIVE`: 활성 구독 상태

이러한 상태 값들은 구매의 다양한 단계와 구독 상태를 나타냅니다.
